### Monta Tabela de Anotação

In [1]:
import collections
import re
import pandas as pd

dictF = collections.OrderedDict()
anotador = 0
cont = 1
str1 = ""

for filename in ['Clear_07_3_ISO_Andressa_Bressane.xml','Clear_07_3_ISO_Eric_Eltz.xml', 'Clear_07_3_ISO_Keli_Schell.xml' ]:
    
    with open(filename,'r') as file:
        cont = 1
        for line in file:

            if line.find("EM-ID") >= 0 and line.find("/EM") >= 0:
                str1 = re.findall('(?<=>).*?(?=<)', line)[0] + " - " + str(cont)
                categoria = re.findall('(?<=CATEG=").*?(?=")', line)[0]
                #print(str1)
                if str1 in dictF:
                    dictF[str1][anotador] = categoria
                else:
                    dictF[str1] = ['','','']
                    dictF[str1][anotador] = categoria
            
            elif line.find("EM-ID") >= 0:
                str1 = re.findall('(?<=>).*?(?=$)', line)[0]
                str1 = str1.replace("\n", "")
                categoria = re.findall('(?<=CATEG=").*?(?=")', line)[0]
            
            elif line.find("/EM") >= 0:
                str1 = str1 + " " + re.findall('(?<=^).*?(?=<)', line)[0] + " - " + str(cont)
                str1 = str1.replace("\n", "")
                #print(str1)
                if str1 in dictF:
                    dictF[str1][anotador] = categoria
                else:
                    dictF[str1] = ['','','']
                    dictF[str1][anotador] = categoria
                str1 = ""
            
            else:
                str1 = str1 + " " + line
                
            cont += 1
        
    anotador += 1
    
#print(len(dictF))
    
kappa_pd = pd.DataFrame.from_dict(dictF,orient='index')
kappa_pd.sample(10)

,0,1,2
Formação Salitre - 922,unidadeESTRATIGRAFICA,unidadeESTRATIGRAFICA,unidadeESTRATIGRAFICA
cinturão de antepaís - 3489,CONTEXTOgeologicoDeBACIA,,
tectônica - 3427,,,CONTEXTOgeologicoDeBACIA
cinturão epidérmico de antepaís - 109,CONTEXTOgeologicoDeBACIA,,
Bacia de Irecê - 3466,baciaSEDIMENTAR,baciaSEDIMENTAR,baciaSEDIMENTAR
cinturão epidérmico de antepaís - 3143,CONTEXTOgeologicoDeBACIA,,
Empurrões - 4201,,CONTEXTOgeologicoDeBACIA,
Bacia de Irecê - 1416,baciaSEDIMENTAR,baciaSEDIMENTAR,baciaSEDIMENTAR
deformação - 2146,,,CONTEXTOgeologicoDeBACIA
580 e 540 Ma - 4186,idade,,idade


In [2]:
all_3 = kappa_pd[(kappa_pd[0]!="") & (kappa_pd[1]!="") & (kappa_pd[2]!="")]
print(all_3.shape)
all_3

(72, 3)


,0,1,2
Bacia de Irecê - 13,baciaSEDIMENTAR,baciaSEDIMENTAR,baciaSEDIMENTAR
neoproterozoicas - 21,era,eon,era
Bacia de Irecê - 113,baciaSEDIMENTAR,baciaSEDIMENTAR,baciaSEDIMENTAR
Bacia de Irecê - 233,baciaSEDIMENTAR,baciaSEDIMENTAR,baciaSEDIMENTAR
Grupo Una - 281,unidadeESTRATIGRAFICA,unidadeESTRATIGRAFICA,unidadeESTRATIGRAFICA
Supergrupo Espinhaço - 288,unidadeESTRATIGRAFICA,unidadeESTRATIGRAFICA,unidadeESTRATIGRAFICA
Bacia de Irecê - 294,baciaSEDIMENTAR,baciaSEDIMENTAR,baciaSEDIMENTAR
Bacia de Irecê - 324,baciaSEDIMENTAR,baciaSEDIMENTAR,baciaSEDIMENTAR
Grupo Una - 399,unidadeESTRATIGRAFICA,unidadeESTRATIGRAFICA,unidadeESTRATIGRAFICA
Pré-cambriano - 506,EON,EON,EON


In [2]:
classes = set()

for x in kappa_pd[0].unique():
    classes.add(x.lower())
    
for x in kappa_pd[1].unique():
    classes.add(x.lower())
    
for x in kappa_pd[2].unique():
    classes.add(x.lower())

classes

{'',
 'baciasedimentar',
 'contextogeologicodebacia',
 'eon',
 'era',
 'idade',
 'outro',
 'periodo',
 'sedimentarescarbonaticas',
 'sedimentaresquimicas',
 'sedimentaressiliciclasticas',
 'unidadeestratigrafica'}

### Montagem da Tabela para o Kappa

In [3]:
w, h = len(classes), len(kappa_pd);
kappa_table = [[0 for x in range(w)] for y in range(h)] 

row_id = 0
for index, row in kappa_pd.iterrows():
    class_id = 0
    for clas in classes:
        if row[0].lower() == clas:
            kappa_table[row_id][class_id] += 1
        if row[1].lower() == clas:
            kappa_table[row_id][class_id] += 1
        if row[2].lower() == clas:
            kappa_table[row_id][class_id] += 1
        class_id += 1
    row_id += 1
            
kappa_table[:10]

[[0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0],
 [0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
 [0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
 [0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0]]

In [4]:
from statsmodels.stats.inter_rater import fleiss_kappa
fleiss_kappa(kappa_table)

0.16027657450019131

### Calculo Kappa de Identificação

In [5]:
w, h = 2, len(kappa_pd);
kappa_id_table = [[0 for x in range(w)] for y in range(h)]

row_id = 0
for index, row in kappa_pd.iterrows():
    class_id = 0
    for clas in classes:
        if row[0].lower() == clas:
            kappa_id_table[row_id][class_id] += 1
        if row[1].lower() == clas:
            kappa_id_table[row_id][class_id] += 1
        if row[2].lower() == clas:
            kappa_id_table[row_id][class_id] += 1
        if class_id == 0:
            class_id += 1
    row_id += 1
    
kappa_id_table[:10]

[[0, 3],
 [0, 3],
 [0, 3],
 [0, 3],
 [0, 3],
 [0, 3],
 [0, 3],
 [0, 3],
 [0, 3],
 [0, 3]]

In [6]:
fleiss_kappa(kappa_id_table)

-0.0016051364366164906

### Calculo Kappa da Classificação

In [7]:
kappa_class_table = pd.DataFrame(data=kappa_table)

print(kappa_class_table.shape)

class_id = 0
for clas in classes:
    print("")
    print(str(class_id) + clas)
    kappa_class_table = kappa_class_table[kappa_class_table[class_id]==0]
    kappa_class_table.drop(kappa_class_table.columns[class_id],inplace=True,axis=1)
    print(kappa_class_table.shape)
    print(fleiss_kappa(kappa_class_table.as_matrix()))
    kappa_class_table = pd.DataFrame(data=kappa_table)
    class_id += 1

(416, 12)

0sedimentaresquimicas
(414, 11)
0.160957112971

1
(72, 11)
0.885573017835

2sedimentaressiliciclasticas
(402, 11)
0.134224022809

3baciasedimentar
(382, 11)
0.0324081131033

4outro
(406, 11)
0.163771810068

5unidadeestratigrafica
(377, 11)
0.085311535091

6contextogeologicodebacia
(133, 11)
0.565895188533

7era
(399, 11)
0.1493689721

8idade
(412, 11)
0.158823710097

9sedimentarescarbonaticas
(399, 11)
0.154415176997

10eon
(401, 11)
0.131557924432

11periodo
(415, 11)
0.156381303762


In [24]:
kappa_class_table = pd.DataFrame(data=kappa_table)
kappa_class_table = kappa_class_table[kappa_class_table[1]==0]
kappa_class_table.drop(kappa_class_table.columns[1],inplace=True,axis=1)

print(kappa_class_table.shape)

class_id = 0
for clas in classes:
    if class_id != 1: 
        print("")
        print(str(class_id) + clas)
        kappa_class_table = kappa_class_table[kappa_class_table[class_id]==0]
        kappa_class_table.drop(kappa_class_table.columns[class_id],inplace=True,axis=1)
        print(kappa_class_table.shape)
        print(fleiss_kappa(kappa_class_table.as_matrix()))
        
    kappa_class_table = pd.DataFrame(data=kappa_table)
    kappa_class_table = kappa_class_table[kappa_class_table[1]==0]
    kappa_class_table.drop(kappa_class_table.columns[1],inplace=True,axis=1)
    class_id += 1

(72, 11)

0sedimentaresquimicas
(72, 10)
0.885573017835

2sedimentaressiliciclasticas
(65, 10)


AssertionError: 